In [1]:
#Systematic Methods of Chemical Process Design
# 16.2 SEQUENTIAL SYNTHESIS
# 16.2.3 Prediction of Matches for Minimizing the Number of Units
# The Solution of the MILP transshipment problem will indicate the matches that take place and the heat exchanged at each match

import numpy as np
import pandas as pd
import pyomo.environ as pyo

In [2]:
# The data is taken from example 16.4
# At this point, we will already have solved for the utility requirements. Last stream of the hot and cold streams are utility streams
# Fcp - heat capacity
# Tin - initial temperature in K
# Tout - final temperature in K
# tmapp - approach temperature

hot_streams = pd.DataFrame(data=[[1,400,120],[2,340,120],
                                 # utility stream
                                 [600,500,499.9]],
                       index=[1,2,3],
                       columns=['Fcp','Tin', 'Tout'])

cold_streams = pd.DataFrame(data=[[1.5,160,400],[1.3,100,250], 
                                  #utility stream
                                  [22.5,20,30]],
                       index=[1,2,3],
                       columns=['Fcp','Tin', 'Tout'])

tmapp = 20

In [3]:
#Temperature Intervals

# Temperatures of hot and cold streams are ordered from highest to lowest and duplicate values are dropped
T_hot  = np.unique(hot_streams[['Tin','Tout']])
T_cold = np.unique(cold_streams[['Tin','Tout']])

T_intervals = pd.Series(data = np.unique(np.concatenate((T_hot,T_cold)))[::-1])
T_intervals.index += 1

#Shifted Temperatures - temperatures are shifted by +/- tmapp
T_shifted = pd.Series(data = np.unique(np.concatenate((T_hot-tmapp/2,T_cold+tmapp/2)))[::-1])
T_shifted.index += 1

In [4]:
# Determine which streams are present at each interval
i_hot = pd.Series(data = ((hot_streams['Tin']-tmapp/2 >= T) & (hot_streams['Tout']-tmapp/2 <= Tnext) for T,Tnext in zip(T_shifted,T_shifted[1:])))
i_hot.index += 1

i_cold = pd.Series(data = ((cold_streams['Tout']+tmapp/2 >= T) & (cold_streams['Tin']+tmapp/2 <= Tnext) for T,Tnext in zip(T_shifted,T_shifted[1:])))
i_cold.index += 1

In [5]:
model = pyo.ConcreteModel()

# Sets
model.k  = pyo.RangeSet(0,len(T_shifted))      # interval k
model.st = pyo.RangeSet(1,len(T_shifted)-1)    # stage st 
model.i  = pyo.RangeSet(1,len(hot_streams))    # hot stream i
model.j  = pyo.RangeSet(1,len(cold_streams))   # cold stream j

# Variables
model.Q      = pyo.Var(model.i, model.j, model.st, domain=pyo.NonNegativeReals, initialize = 0)   # heat exchanged between i and j at interval st
model.r      = pyo.Var(model.i, model.k,domain=pyo.NonNegativeReals, initialize = 0)              # heat residual at interval k

# Binary Variables
model.y = pyo.Var(model.i, model.j, domain=pyo.Binary)     # match that takes place between hot stream i and cold stream j

# Parameters - Upper bounds for heat exchange
def bounds(model,i,j):
    hot  = hot_streams['Fcp'][i]*(hot_streams['Tin'][i] - hot_streams['Tout'][i])
    cold = cold_streams['Fcp'][j]*(cold_streams['Tout'][j] - cold_streams['Tin'][j])
    return min(hot,cold)
    

model.U = pyo.Param(model.i, model.j, domain=pyo.NonNegativeReals, initialize = bounds)

# Constraints
# Total heat exchanged by hot stream i
def bal_hot_stream(model,i,k):
    return model.r[i,k] - model.r[i,k-1] \
                          + sum(model.Q[i,j,k] for j in model.j)\
                          == hot_streams['Fcp'][i]*(T_shifted[k] - T_shifted[k+1])*int(i_hot[k][i])

model.cons_hot = pyo.Constraint(model.i, model.st, rule = bal_hot_stream)

# Energy exchanged by cold stream j in stage k
def bal_cold_stream(model,j,k):
    if i_cold[k][j]:
        return sum(model.Q[i,j,k] for i in model.i) == cold_streams['Fcp'][j]*(T_shifted[k] - T_shifted[k+1])
    return pyo.Constraint.Skip

model.cons_cold = pyo.Constraint(model.j,model.st, rule = bal_cold_stream)

# First and last residual constraint
def res_0(model,i):
    return  model.r[i,0] == 0

def res_k(model,i):
    return  model.r[i,len(model.st)] == 0

model.first_res = pyo.Constraint(model.i,rule = res_0)
model.last_res = pyo.Constraint(model.i,rule = res_k)

# Define constrained matches
# def constrained_matches(model,k):
#     #specify streams
#     return model.Q[1,1,k] == 0

# model.cmatches = pyo.Constraint(model.st, rule = constrained_matches)

# Minimum number of units above and below the pinch
def units(model,i,j):
    return sum(model.Q[i,j,k] for k in model.st) - model.U[i,j]*model.y[i,j] <= 0

model.cons_y = pyo.Constraint(model.i, model.j, rule = units)

model.OBJ = pyo.Objective(expr = pyo.summation(model.y), sense = pyo.minimize)
# Results
results = pyo.SolverFactory("cplex").solve(model, tee = False)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: tmpu11l_08q
  Lower bound: 5.0
  Upper bound: 5.0
  Number of objectives: 1
  Number of constraints: 50
  Number of variables: 121
  Number of nonzeros: 253
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: 0.0
  Termination condition: optimal
  Termination message: MIP - Integer optimal solution\x3a Objective = 5.0000000000e+00
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.12456607818603516
# --------------

In [6]:
# Matches Results
model.y.pprint()

y : Size=9, Index=y_index
    Key    : Lower : Value : Upper : Fixed : Stale : Domain
    (1, 1) :     0 :   1.0 :     1 : False : False : Binary
    (1, 2) :     0 :   1.0 :     1 : False : False : Binary
    (1, 3) :     0 :  -0.0 :     1 : False : False : Binary
    (2, 1) :     0 :   1.0 :     1 : False : False : Binary
    (2, 2) :     0 :  -0.0 :     1 : False : False : Binary
    (2, 3) :     0 :   1.0 :     1 : False : False : Binary
    (3, 1) :     0 :   1.0 :     1 : False : False : Binary
    (3, 2) :     0 :   0.0 :     1 : False : False : Binary
    (3, 3) :     0 :   0.0 :     1 : False : False : Binary


In [7]:
# Heat exchanged between matches
for i in model.i:
    for j in model.j:
        print("H", i, "C", j, sum(model.Q[i,j,st].value for st in model.st))

H 1 C 1 85.0
H 1 C 2 195.0
H 1 C 3 0.0
H 2 C 1 215.0
H 2 C 2 0.0
H 2 C 3 225.0
H 3 C 1 60.00000000001368
H 3 C 2 0.0
H 3 C 3 0.0


In [8]:
model.pprint()

7 Set Declarations
    Q_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     3 : i*j*st :   81 : {(1, 1, 1), (1, 1, 2), (1, 1, 3), (1, 1, 4), (1, 1, 5), (1, 1, 6), (1, 1, 7), (1, 1, 8), (1, 1, 9), (1, 2, 1), (1, 2, 2), (1, 2, 3), (1, 2, 4), (1, 2, 5), (1, 2, 6), (1, 2, 7), (1, 2, 8), (1, 2, 9), (1, 3, 1), (1, 3, 2), (1, 3, 3), (1, 3, 4), (1, 3, 5), (1, 3, 6), (1, 3, 7), (1, 3, 8), (1, 3, 9), (2, 1, 1), (2, 1, 2), (2, 1, 3), (2, 1, 4), (2, 1, 5), (2, 1, 6), (2, 1, 7), (2, 1, 8), (2, 1, 9), (2, 2, 1), (2, 2, 2), (2, 2, 3), (2, 2, 4), (2, 2, 5), (2, 2, 6), (2, 2, 7), (2, 2, 8), (2, 2, 9), (2, 3, 1), (2, 3, 2), (2, 3, 3), (2, 3, 4), (2, 3, 5), (2, 3, 6), (2, 3, 7), (2, 3, 8), (2, 3, 9), (3, 1, 1), (3, 1, 2), (3, 1, 3), (3, 1, 4), (3, 1, 5), (3, 1, 6), (3, 1, 7), (3, 1, 8), (3, 1, 9), (3, 2, 1), (3, 2, 2), (3, 2, 3), (3, 2, 4), (3, 2, 5), (3, 2, 6), (3, 2, 7), (3, 2, 8), (3, 2, 9), (3, 3, 1), (3, 3, 2), (3, 3, 3), (3, 3, 4), (3, 3, 5), 